In [2]:
import keras
import tensorflow as tf
from IPython.display import Image
from matplotlib.image import imread
import os
import numpy as np
from keras import layers
import tqdm
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
from keras import Model, Input, regularizers
from keras.layers import Dense, Conv2D, MaxPool2D, UpSampling2D
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [3]:
path_buildings = "./seg_train/seg_train/buildings"
# path_forest = "./seg_train/seg_train/forest"
# path_glacier = "./seg_train/seg_train/glacier"
# path_mountain = "./seg_train/seg_train/mountain"
# path_sea = "./seg_train/seg_train/sea"
# path_street = "./seg_train/seg_train/street"


In [4]:
#Arrays of image vectors (150 X 150 X 3)
all_images = []

for file in os.listdir(path_buildings):
      img = image.load_img(path_buildings + "/"+file, target_size=(80,80,3))
      img = image.img_to_array(img)
      img = img/255.
      all_images.append(img)


# buildings = [imread(path_buildings + "/" + file) for file in os.listdir(path_buildings)]
#buildings = np.asarray(buildings)
# forest = [imread(path_forest + "/" + file) for file in os.listdir(path_forest)]
# #forest = np.asarray(forest)
# glacier = [imread(path_glacier + "/" + file) for file in os.listdir(path_glacier)]
# #glacier = np.asarray(glacier)
# mountain = [imread(path_mountain + "/" + file) for file in os.listdir(path_mountain)]
# #mountain = np.asarray(mountain)
# sea = [imread(path_sea + "/" + file) for file in os.listdir(path_sea)]
# #sea = np.asarray(buildings)
# street = [imread(path_street + "/" + file) for file in os.listdir(path_street)]
#street = np.asarray(street)



In [5]:
all_images = np.array(all_images)

In [6]:
train_x, val_x = train_test_split(all_images, random_state=32, test_size=0.1)

In [7]:
#Model Building of an image


Input_img = keras.Input(shape=(80, 80, 3)) 
x1 = Conv2D(256, (3, 3), activation='relu', padding='same')(Input_img)
x2 = Conv2D(128, (3, 3), activation='relu', padding='same')(x1)
x2 = MaxPool2D( (2, 2))(x2)
encoded = Conv2D(64, (3, 3), activation='softmax', padding='same')(x2)

In [8]:
#Decoding Architecture

x3 = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
x3 = UpSampling2D((2, 2))(x3)
x2 = Conv2D(128, (3, 3), activation='relu', padding='same')(x3)
x1 = Conv2D(256, (3, 3), activation='relu', padding='same')(x2)
decoded = Conv2D(3, (3, 3), padding='same')(x1)

In [9]:
autoencoder = Model(Input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mse')


In [10]:
autoencoder.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 80, 80, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 80, 80, 256)       7168      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 80, 80, 128)       295040    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 40, 40, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 40, 40, 64)        73792     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 40, 40, 64)        36928     
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 80, 80, 64)        0   

In [ ]:
early_stopper = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=4, verbose=1, mode='auto')
a_e = autoencoder.fit(train_x, train_x,
            epochs=10,
            batch_size=256,
            shuffle=True,
            validation_data=(val_x, val_x),
            callbacks=[early_stopper])


Train on 1971 samples, validate on 220 samples
Epoch 1/10
